In [41]:
import pandas as pd
import numpy as np

# Models to use
import lightgbm as lgb
import catboost as cb

# Importing the metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import plot_confusion_matrix

# For measuring the training time taken during the fit process
import time

from hyperopt import hp
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials, space_eval

# Importing the Scalers
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [42]:
df = pd.read_csv('higgs_cleaned.csv')
df.head()

,class,lepton_pT,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet1pt,jet1eta,jet1phi,jet1b-tag,...,jet4eta,jet4phi,jet4b-tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
0,1,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,...,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
1,1,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282322,1.381664,0.000000,...,1.128848,0.900461,0.000000,0.909753,1.108330,0.985692,0.951331,0.803252,0.865924,0.780118
2,0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,...,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
3,1,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,...,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808487
4,0,1.595839,-0.607811,0.007075,1.818450,-0.111906,0.847550,-0.566437,1.581239,2.173076,...,-0.654227,-1.274345,3.101961,0.823761,0.938191,0.971758,0.789176,0.430553,0.961357,0.957818


In [43]:
X, y = df.drop('class', axis=1), df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1864)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1864)

In [44]:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train.values.reshape(-1, 1)).reshape(-1)
y_val = scaler.transform(y_val.values.reshape(-1, 1)).reshape(-1)
y_test = scaler.transform(y_test.values.reshape(-1, 1)).reshape(-1)

Scaling is surely an important part of the pipeline but I realized that I forgot doing it in the end. I will add several choices for scalers later on, but for now lets move to designing the search space.

For the meaning of search functions such as hp.uniform, you can take a look at here: http://hyperopt.github.io/hyperopt/getting-started/search_spaces/.

In [45]:
classifier_parameters = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.01, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(2, 13, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(0.5, 10, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.5, 1, 0.1)),
    'subsample':        hp.uniform('subsample', 0.6, 1),
    'n_estimators':     hp.choice('n_estimators ', [100, 150, 200, 350, 500]),
    'min_child_samples':hp.choice('min_child_samples', np.arange(50, 500, 75, dtype=int)),
    'num_leaves':       hp.choice('num_leaves', [4, 8, 16, 32]),
    'objective':        'binary',
    'boosting_type':    hp.choice('boosting_type', ['gbdt', 'dart']),
    'random_seed':      1864
}
fit_parameters = {
    'eval_metric': 'logloss',
    'early_stopping_rounds': 10,
    'verbose': False
}

lgb_para = dict()
lgb_para['clf_params'] = classifier_parameters
lgb_para['fit_params'] = fit_parameters

In [46]:
classifier_parameters = {
    'l2_leaf_reg': hp.qloguniform('l2_leaf_reg', 0, 2, 1), # Coefficient of the regularizer for the cost function.
    'learning_rate': hp.uniform('learning_rate', 1e-3, 5e-1), # Self-explanatory. I think this will be quite important since it affects overfitting
    'max_depth': hp.choice('max_depth', np.arange(2, 14, dtype=int)), # Tree depths
    'num_trees': hp.choice('num_trees', [25, 50, 100, 150 , 200, 350, 500]), # Number of trees to be grown
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1.0), # Fraction of randomly selected features to use at each split
    'min_data_in_leaf': hp.choice('min_data_in_leaf', np.arange(2, 32, 2, dtype=int)), # Minimum data required in leaves for a split to occur
    #'num_leaves': hp.choice('num_leaves', [4, 8, 16, 32]), # max number of leaves
    'eval_metric': 'Accuracy',
    'loss_function':'Logloss',
    'random_seed': 1864
}


fit_parameters = {
    'early_stopping_rounds': 10,
    'verbose': False
}

ctb_para = dict()
ctb_para['clf_params'] = classifier_parameters
ctb_para['fit_params'] = fit_parameters

Objective function to optimize. It is a bit wordy for now but defining it as a class rather than a function may come in handy later on, in case we add more boosting models.

In [47]:
class objective_fn(object):
    
    def __init__(self, X_train, X_val, X_test, y_train, y_val, y_test):
        self.X_train = X_train
        self.X_val   = X_val
        self.X_test  = X_test
        
        self.y_train = y_train
        self.y_val   = y_val
        self.y_test  = y_test
        
    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials
    
    def ctb_clf(self, para):
        clf = cb.CatBoostClassifier(**para['clf_params'])
        return self.train(clf, para)
    
    def lgb_clf(self, para):
        clf = lgb.LGBMClassifier(**para['clf_params'])
        return self.train(clf, para)
    
    def train(self, clf, para):
        clf.fit(self.X_train, self.y_train,
                eval_set=[(self.X_train, self.y_train), (self.X_val, self.y_val)],
                **para['fit_params'])
        
        preds = clf.predict(self.X_test)
        acc = accuracy_score(self.y_test, preds)
        return {'loss': -acc, 'status': STATUS_OK}

We can now start the trials. Hyperopt tries to minimize the objective function value so the more negative the accuracy, the better. Playing with clf parameters or the value of maximum number of evaluations may give better results, but I'll keep the searching procedure not too expensive.

In [48]:
obj = objective_fn(X_train, X_val, X_test, y_train, y_val, y_test)

trials = Trials()
start = time.time()
best = obj.process(fn_name='lgb_clf', space=lgb_para, trials=trials, algo=tpe.suggest, max_evals=200)
end = time.time()

total_time = end - start

  0%|                                                                                                                                                       | 0/200 [00:00<?, ?trial/s, best loss=?]

C:\Users\canberk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\lightgbm\callback.py:186: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



  0%|▌                                                                                                                         | 1/200 [00:23<1:16:21, 23.02s/trial, best loss: -0.6774604793472718]

C:\Users\canberk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\lightgbm\callback.py:186: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



  1%|█▏                                                                                                                          | 2/200 [00:28<58:53, 17.85s/trial, best loss: -0.7146353901070882]

...

 98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏ | 197/200 [1:29:10<01:32, 30.85s/trial, best loss: -0.7306986231514533]

C:\Users\canberk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\lightgbm\callback.py:186: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 199/200 [1:29:20<00:17, 17.54s/trial, best loss: -0.7306986231514533]

C:\Users\canberk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\lightgbm\callback.py:186: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [1:30:18<00:00, 27.09s/trial, best loss: -0.7306986231514533]


In [49]:
print('Parameters for the best LGBM Model:')
print()
for k,v in best[0].items():
    print(k, ':', v)
print()
print('Time taken for HyperParam Search:', round(total_time/60, 2), 'mins')

Parameters for the best LGBM Model:

boosting_type : 1
colsample_bytree : 3
learning_rate : 4
max_depth : 10
min_child_samples : 1
min_child_weight : 8
n_estimators  : 4
num_leaves : 3
subsample : 0.7049265682103403

Time taken for HyperParam Search: 90.31 mins


We have used the hp.choice function which returns the **index** of the best hyperparameter inside the given list of hyperparameters. So of course, the best hyperparameters for the number of trees is not 2, but 100.

The list below returns the correct model hyperparameters.

In [50]:
best_params = space_eval(lgb_para['clf_params'], trials.argmin)
best_params

{'boosting_type': 'dart',
 'colsample_bytree': 0.7999999999999999,
 'learning_rate': 0.21000000000000002,
 'max_depth': 12,
 'min_child_samples': 125,
 'min_child_weight': 8,
 'n_estimators': 500,
 'num_leaves': 32,
 'objective': 'binary',
 'random_seed': 1864,
 'subsample': 0.7049265682103403}

In [51]:
train_new_x = pd.concat([pd.DataFrame(X_train), pd.DataFrame(X_val)])
train_new_y = pd.concat([pd.DataFrame(y_train.reshape(-1)), pd.DataFrame(y_val.reshape(-1))])

In [52]:
clf = lgb.LGBMClassifier(**best_params)

clf.fit(train_new_x, train_new_y, verbose=False, eval_set=[(train_new_x, train_new_y), (X_test, y_test.reshape(-1))],
        eval_metric= 'logloss',
        early_stopping_rounds=50)

preds = clf.predict(X_test)

print(accuracy_score(y_test.reshape(-1), preds))

C:\Users\canberk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\canberk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\lightgbm\callback.py:186: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


0.7283528811830698


In [54]:
clf.booster_.save_model('model_minmax_lgbm_dart.txt')

In [53]:
"""clf.save_model(
    "model3_minmax_lgbm_dart.json",
    format="json",
    # pool=pool  # this parameter is required only for models with categorical features.
)"""

AttributeError: 'LGBMClassifier' object has no attribute 'save_model'